source :

https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/

https://cs231n.github.io/linear-classify/

https://peterroelants.github.io/posts/cross-entropy-softmax/
# Softmax Function 

The **logistic function** can only be used for classification between two target classes $t=0 \text{ and }  t=1$. This logistic function cna be generalized to output a multiclass categorical probability distribution by **Softmax function**. This softmax function $\zeta$ takes a $C$ dimensional  vector $z$ and outputs a $C-$dimensional vector of $y$ of real values between $[0,1]$.This function is a normalized exponential and is defined as:

\begin{align}
y_c = \zeta(z)_c = \frac{e^{z_c}}{\sum_{d=1}^C e^{z_d}} \text{ for } c =1,\ldots,C
\end{align}

The denominator $\sum_{d=1}^C e^{z_d}$ acts as a regulizer to make sure that $\sum_{c=1}^C y_c =1$. As the output layer of a Neural Network, the softmax function can be represented graphically as a layer with $C$ neurons.
 We can write the probabilities that the class $t=c \text{ for }  c=1,\ldots,C$ given input $z$ as:

\begin{gather}
  \begin{bmatrix} 
  P(t = 1/z) \\     
  \vdots \\      
  P(t = C/z)  \\
  \end{bmatrix} 
  = 
  \begin{bmatrix} 
  \zeta(z)_1 \\     
  \vdots \\      
  \zeta(z)_C \\
  \end{bmatrix} 
  =
  \frac{1}{\sum_{d=1}^C e^{z_d}}
  \begin{bmatrix} 
  e^{z_1} \\     
  \vdots \\      
  e^{z_C} \\
  \end{bmatrix}
\end{gather}

Where $P(t=c/z)$ is thus the probability that that the class is $c$ given by the input $z$.


# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
import random
import numpy as np
import math
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
#%load_ext autoreload
#%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = cifar10.load_data()
    
    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    #print("X_train length",len(X_train))
    #print("y_train length",len(y_train))
    #print("X_test length",len(X_test))
    #print("y_test length",len(y_test))

    
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train = X_train.astype('float32')
    X_train -= mean_image
    X_val = X_val.astype('float32')
    X_val -= mean_image
    X_test = X_test.astype('float32')
    X_test -= mean_image
    X_dev = X_dev.astype('float32')
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

170508288/170498071 [==============================] - 4s 0us/step
Train data shape:  (49000, 3073)
Train labels shape:  (49000, 1)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000, 1)
Test data shape:  (1000, 3073)
Test labels shape:  (1000, 1)
dev data shape:  (500, 3073)
dev labels shape:  (500, 1)


In [6]:
import numpy as np
from random import shuffle

def softmax_loss_naive(W, X, y, reg):
  """
  Softmax loss function, naive implementation (with loops)
  Inputs have dimension D, there are C classes, and we operate on minibatches
  of N examples.
  Inputs:
  - W: A numpy array of shape (D, C) containing weights.
  - X: A numpy array of shape (N, D) containing a minibatch of data.
  - y: A numpy array of shape (N,) containing training labels; y[i] = c means
    that X[i] has label c, where 0 <= c < C.
  - reg: (float) regularization strength
  Returns a tuple of:
  - loss as single float
  - gradient with respect to weights W; an array of same shape as W
  """
  # Initialize the loss and gradient to zero.
  N, D = X.shape
  C = W.shape[1]
  num_classes = W.shape[1]
  loss = 0.0
  out = np.zeros((N,C))
  dW = np.zeros_like(W)  # (3073, 10)
  
  #############################################################################
  # TODO: Compute the softmax loss and its gradient using explicit loops.     #
  # Store the loss in loss and the gradient in dW. If you are not careful     #
  # here, it is easy to run into numeric instability. Don't forget the        #
  # regularization!                                                           #
  #############################################################################
  
  
  z = np.dot(X,W)
  softmax_out = np.zeros(z.shape[0])
  sum_exp = np.sum(np.exp(z),axis = 1)

  # Softmax function double loop 
  for i in range(0,z.shape[0]):
    for j in range(0,z.shape[1]):

      softmax_out[i] = - math.log(np.divide(np.exp(z[i,y[i]]),sum_exp[i]))
    
  # loss 
  
  loss = np.sum(softmax_out)
  
  #gradient

  grad_z = np.zeros(z.shape)
  for i in range(0,z.shape[0]):
    for j in range(0,z.shape[1]):
      if y[i] == j:

        y_hat = np.divide(np.exp(z[i,j]),sum_exp[i])
        grad_z[i,j] = y_hat*(1-y_hat)
   
      else:
        y_hat_1 = np.divide(np.exp(z[i,y[i]]),sum_exp[i])
        y_hat_2 = np.divide(np.exp(z[i,j]),sum_exp[i])
        grad_z[i,j] = - y_hat_1*y_hat_2 
      
      grad_z[i,j] *=  - np.divide(sum_exp[i],np.exp(z[i,y[i]]))

  dW = np.dot(grad_z.T,X)

  loss /= N
  loss += reg * np.sum(W * W)
  dW /= N
  dW = dW.T
  dW += 2 * reg * W
 
  #############################################################################
  #                          END OF YOUR CODE                                 #
  #############################################################################

  return loss, dW


def softmax_loss_vectorized(W, X, y, reg):
  """
  Softmax loss function, vectorized version.
  Inputs and outputs are the same as softmax_loss_naive.
  """
  # Initialize the loss and gradient to zero.
  N = X.shape[0]
  loss = 0.0
  dW = np.zeros_like(W)
 
  #############################################################################
  # TODO: Compute the softmax loss and its gradient using no explicit loops.  #
  # Store the loss in loss and the gradient in dW. If you are not careful     #
  # here, it is easy to run into numeric instability. Don't forget the        #
  # regularization!                                                           #
  #############################################################################
   
  # vectorized loss 

  z = np.dot(X,W)

  # Initializing the Softmax_loss table
  softmax_loss = np.zeros(z.shape[0])

  # selecting the score of the true class for each entry 
  true_class_exp = np.exp(np.take(z,y)).reshape(z.shape[0],1)

  # calculating exponential of individual values of the score table
  z_exponential = np.exp(z)

  z_exponential_sum = np.sum(z_exponential,axis=1).reshape(z.shape[0],1)

  #softmax_loss = - np.log(np.divide(true_class_exp,z_exponential_sum))

  loss = -np.sum(np.log(true_class_exp/z_exponential_sum))


  # vectorized gradient

  z_exp_table = np.divide(np.exp(z),z_exponential_sum)

  true_class_exp_normalized = np.divide(true_class_exp,z_exponential_sum)

  z_exp_table *= (-1)*true_class_exp_normalized
   #- (scores_sums - cors) / scores_sums

  z_exp_table[np.arange(0,z.shape[0]),y] = -(z_exponential_sum - true_class_exp)/z_exponential_sum

  z_exp_table*= (-1)*np.divide(z_exponential_sum,true_class_exp)
 
  dW = np.dot(z_exp_table.T,X)
  print(dW.shape)
  


  loss /= N
  loss += reg * np.sum(W * W)
  dW /= N
  dW = dW.T
  dW += 2 * reg * W
    
  #############################################################################
  #                          END OF YOUR CODE                                 #
  #############################################################################

  return loss, dW

## Softmax Classifier

Your code for this section will all be written inside `cs231n/classifiers/softmax.py`.


In [4]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

#from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.370488
sanity check: 2.302585


**Inline Question 1**

Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

$\color{blue}{\textit Your Answer:}$ *Fill this in* 



In [5]:
from random import randrange

def grad_check_sparse(f, x, analytic_grad, num_checks=10, h=1e-5):
    """
  sample a few random elements and only return numerical
  in this dimensions.
  """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (abs(grad_numerical) + abs(grad_analytic))
        print('numerical: %f analytic: %f, relative error: %e' % (grad_numerical, grad_analytic, rel_error))

In [ ]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
#from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 1.413832 analytic: 1.413832, relative error: 6.118107e-09
numerical: -5.813914 analytic: -5.813914, relative error: 3.120527e-09
numerical: 0.546501 analytic: 0.546501, relative error: 5.819260e-08
numerical: 2.894648 analytic: 2.894648, relative error: 5.182705e-10
numerical: -1.248803 analytic: -1.248803, relative error: 4.075285e-08
numerical: 1.585313 analytic: 1.585313, relative error: 3.762081e-08
numerical: 2.198136 analytic: 2.198136, relative error: 1.505888e-08
numerical: -0.595038 analytic: -0.595038, relative error: 9.756414e-08
numerical: -0.965683 analytic: -0.965683, relative error: 1.648608e-08
numerical: -0.628681 analytic: -0.628681, relative error: 3.997133e-08
numerical: 2.873328 analytic: 2.873328, relative error: 1.046334e-08
numerical: 1.202751 analytic: 1.202751, relative error: 2.255219e-08
numerical: -1.302305 analytic: -1.302305, relative error: 2.634548e-08
numerical: -3.389847 analytic: -3.389847, relative error: 1.136095e-08
numerical: 0.088412 

In [ ]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

#from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.404715e+00 computed in 0.196907s
(10, 3073)
vectorized loss: 2.445080e+00 computed in 0.014807s
Loss difference: 0.040365
Gradient difference: 13453.152269


In [1]:
import numpy as np
#from cs231n.classifiers.linear_svm import *
#from cs231n.classifiers.softmax import *

class LinearClassifier(object):

  def __init__(self):
    self.W = None

  def train(self, X, y, learning_rate=1e-3, reg=1e-5, num_iters=100,
            batch_size=200, verbose=False):
    """
    Train this linear classifier using stochastic gradient descent.
    Inputs:
    - X: A numpy array of shape (N, D) containing training data; there are N
      training samples each of dimension D.
    - y: A numpy array of shape (N,) containing training labels; y[i] = c
      means that X[i] has label 0 <= c < C for C classes.
    - learning_rate: (float) learning rate for optimization.
    - reg: (float) regularization strength.
    - num_iters: (integer) number of steps to take when optimizing
    - batch_size: (integer) number of training examples to use at each step.
    - verbose: (boolean) If true, print progress during optimization.
    Outputs:
    A list containing the value of the loss function at each training iteration.
    """
    num_train, dim = X.shape
    num_classes = np.max(y) + 1 # assume y takes values 0...K-1 where K is number of classes
    if self.W is None:
      # lazily initialize W
      self.W = 0.001 * np.random.randn(dim, num_classes)

    # Run stochastic gradient descent to optimize W
    loss_history = []
    for it in range(num_iters):
      X_batch = None
      y_batch = None

      #########################################################################
      # TODO:                                                                 #
      # Sample batch_size elements from the training data and their           #
      # corresponding labels to use in this round of gradient descent.        #
      # Store the data in X_batch and their corresponding labels in           #
      # y_batch; after sampling X_batch should have shape (dim, batch_size)   #
      # and y_batch should have shape (batch_size,)                           #
      #                                                                       #
      # Hint: Use np.random.choice to generate indices. Sampling with         #
      # replacement is faster than sampling without replacement.              #
      #########################################################################
      
      mask = np.random.choice(num_train,batch_size)
    
      X_batch = X[mask]
      y_batch = y[mask]
        
      #########################################################################
      #                       END OF YOUR CODE                                #
      #########################################################################

      # evaluate loss and gradient
      loss, grad = self.loss(X_batch, y_batch, reg)
      loss_history.append(loss)

      # perform parameter update
      #########################################################################
      # TODO:                                                                 #
      # Update the weights using the gradient and the learning rate.          #
      #########################################################################
      
      self.W -= learning_rate * grad  
        
      #########################################################################
      #                       END OF YOUR CODE                                #
      #########################################################################

      if verbose and it % 100 == 0:
        print ('iteration %d / %d: loss %f' % (it, num_iters, loss))

    return loss_history

  def predict(self, X):
    """
    Use the trained weights of this linear classifier to predict labels for
    data points.
    Inputs:
    - X: D x N array of training data. Each column is a D-dimensional point.
    Returns:
    - y_pred: Predicted labels for the data in X. y_pred is a 1-dimensional
      array of length N, and each element is an integer giving the predicted
      class.
    """
    y_pred = np.zeros(X.shape[1])
    ###########################################################################
    # TODO:                                                                   #
    # Implement this method. Store the predicted labels in y_pred.            #
    ###########################################################################
    
    scores = np.dot(X,self.W)
    y_pred = np.argmax(scores,axis=1)
    
    ###########################################################################
    #                           END OF YOUR CODE                              #
    ###########################################################################
    return y_pred
  
  def loss(self, X_batch, y_batch, reg):
    """
    Compute the loss function and its derivative. 
    Subclasses will override this.
    Inputs:
    - X_batch: A numpy array of shape (N, D) containing a minibatch of N
      data points; each point has dimension D.
    - y_batch: A numpy array of shape (N,) containing labels for the minibatch.
    - reg: (float) regularization strength.
    Returns: A tuple containing:
    - loss as a single float
    - gradient with respect to self.W; an array of the same shape as W
    """
    pass


class LinearSVM(LinearClassifier):
  """ A subclass that uses the Multiclass SVM loss function """

  def loss(self, X_batch, y_batch, reg):
    #return svm_loss_vectorized(self.W, X_batch, y_batch, reg)
    return svm_loss_naive(self.W, X_batch, y_batch, reg)


class Softmax(LinearClassifier):
  """ A subclass that uses the Softmax + Cross-entropy loss function """

  def loss(self, X_batch, y_batch, reg):
    return softmax_loss_vectorized(self.W, X_batch, y_batch, reg)

In [ ]:
import numpy as np
from random import shuffle

def softmax_loss_naive(W, X, y, reg):
  """
  Softmax loss function, naive implementation (with loops)
  Inputs have dimension D, there are C classes, and we operate on minibatches
  of N examples.
  Inputs:
  - W: A numpy array of shape (D, C) containing weights.
  - X: A numpy array of shape (N, D) containing a minibatch of data.
  - y: A numpy array of shape (N,) containing training labels; y[i] = c means
    that X[i] has label c, where 0 <= c < C.
  - reg: (float) regularization strength
  Returns a tuple of:
  - loss as single float
  - gradient with respect to weights W; an array of same shape as W
  """
  # Initialize the loss and gradient to zero.
  N, D = X.shape
  C = W.shape[1]
  num_classes = W.shape[1]
  loss = 0.0
  out = np.zeros((N,C))
  dW = np.zeros_like(W)  # (3073, 10)
  
  #############################################################################
  # TODO: Compute the softmax loss and its gradient using explicit loops.     #
  # Store the loss in loss and the gradient in dW. If you are not careful     #
  # here, it is easy to run into numeric instability. Don't forget the        #
  # regularization!                                                           #
  #############################################################################
  
  
  z = np.dot(X,W)
  softmax_out = np.zeros(z.shape[0])
  sum_exp = np.sum(np.exp(z),axis = 1)

  # Softmax function double loop 
  for i in range(0,z.shape[0]):
    for j in range(0,z.shape[1]):

      softmax_out[i] = - math.log(np.divide(np.exp(z[i,y[i]]),sum_exp[i]))
    
  # loss 
  
  loss = np.sum(softmax_out)
  
  #gradient

  grad_z = np.zeros(z.shape)
  for i in range(0,z.shape[0]):
    for j in range(0,z.shape[1]):
      if y[i] == j:

        y_hat = np.divide(np.exp(z[i,j]),sum_exp[i])
        grad_z[i,j] = y_hat*(1-y_hat)
   
      else:
        y_hat_1 = np.divide(np.exp(z[i,y[i]]),sum_exp[i])
        y_hat_2 = np.divide(np.exp(z[i,j]),sum_exp[i])
        grad_z[i,j] = - y_hat_1*y_hat_2 
      
      grad_z[i,j] *=  - np.divide(sum_exp[i],np.exp(z[i,y[i]]))

  dW = np.dot(grad_z.T,X)

  loss /= N
  loss += reg * np.sum(W * W)
  dW /= N
  dW = dW.T
  dW += 2 * reg * W
 
  #############################################################################
  #                          END OF YOUR CODE                                 #
  #############################################################################

  return loss, dW


def softmax_loss_vectorized(W, X, y, reg):
  """
  Softmax loss function, vectorized version.
  Inputs and outputs are the same as softmax_loss_naive.
  """
  # Initialize the loss and gradient to zero.
  N = X.shape[0]
  loss = 0.0
  dW = np.zeros_like(W)
 
  #############################################################################
  # TODO: Compute the softmax loss and its gradient using no explicit loops.  #
  # Store the loss in loss and the gradient in dW. If you are not careful     #
  # here, it is easy to run into numeric instability. Don't forget the        #
  # regularization!                                                           #
  #############################################################################
   
  # vectorized loss 

  z = np.dot(X,W)

  # Initializing the Softmax_loss table
  softmax_loss = np.zeros(z.shape[0])

  # selecting the score of the true class for each entry 
  true_class_exp = np.exp(np.take(z,y)).reshape(z.shape[0],1)

  # calculating exponential of individual values of the score table
  z_exponential = np.exp(z)

  z_exponential_sum = np.sum(z_exponential,axis=1).reshape(z.shape[0],1)

  #softmax_loss = - np.log(np.divide(true_class_exp,z_exponential_sum))

  loss = -np.sum(np.log(true_class_exp/z_exponential_sum))


  # vectorized gradient

  z_exp_table = np.divide(np.exp(z),z_exponential_sum)

  true_class_exp_normalized = np.divide(true_class_exp,z_exponential_sum)

  z_exp_table *= (-1)*true_class_exp_normalized
   #- (scores_sums - cors) / scores_sums

  z_exp_table[np.arange(0,z.shape[0]),y] = -(z_exponential_sum - true_class_exp)/z_exponential_sum

  z_exp_table*= (-1)*np.divide(z_exponential_sum,true_class_exp)
 
  dW = np.dot(z_exp_table.T,X)
  print(dW.shape)
  


  loss /= N
  loss += reg * np.sum(W * W)
  dW /= N
  dW = dW.T
  dW += 2 * reg * W
    
  #############################################################################
  #                          END OF YOUR CODE                                 #
  #############################################################################

  return loss, dW

In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.

from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################

# Provided as a reference. You may or may not want to change these hyperparameters
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

**Inline Question 2** - *True or False*

Suppose the overall training loss is defined as the sum of the per-datapoint loss over all training examples. It is possible to add a new datapoint to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

$\color{blue}{\textit Your Answer:}$


$\color{blue}{\textit Your Explanation:}$



In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])